# Get GTZAN and MusiCNN-MSD

In [1]:
import os
import tensorflow
import torch
import torchaudio
import numpy as np
import essentia.standard as es
from essentia import Pool
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(42)
###############################################################################################

# Please first specify the path of your GTZAN dataset if it is already downloaded in your system.
    # Otherwise leave 'data' or the desired path where we will download the dataset.

GTZAN_path = 'data'
#GTZAN_path = <your_path>

###############################################################################################

# TensorflowPredictMusiCNN expects mono 16kHz sample rate inputs. Resample needed
resample = es.Resample(inputSampleRate=22050, outputSampleRate=16000, quality=0)

# Download dataset from torchaudio
if not os.path.isdir(GTZAN_path):
    os.mkdir(GTZAN_path)
    train_dataset = torchaudio.datasets.GTZAN(root=GTZAN_path, download=True, subset='training')
else:
    train_dataset = torchaudio.datasets.GTZAN(root=GTZAN_path, subset='training')
val_dataset = torchaudio.datasets.GTZAN(root=GTZAN_path, subset='validation')
test_dataset = torchaudio.datasets.GTZAN(root=GTZAN_path, subset='testing')

# We download the essentia MSD MusiCNN model
if not os.path.isfile('msd-musicnn-1.pb'):
    !curl -SLO https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb

class Essentia_MusiCNNMSD_GTZAN_Dataset(Dataset):
    """ The embeddings of the GTZAN dataset extracted with Essentia-Tensorflow's MusiCNN-MSD model. """
    def __init__(self, GTZAN_dataset, embeddings):
        self.GTZAN_dataset = GTZAN_dataset
        self.embeddings = embeddings
        self.GTZAN_genres = [
            "blues",
            "classical",
            "country",
            "disco",
            "hiphop",
            "jazz",
            "metal",
            "pop",
            "reggae",
            "rock",
        ]
        
    def __len__(self):
        return len(self.GTZAN_dataset)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        inputs = torch.from_numpy(self.embeddings[idx]).mean(0) #comment mean for original method
        labels = torch.tensor(self.GTZAN_genres.index(self.GTZAN_dataset[idx][2]))
        
        return inputs, labels
# We define a shallow model

class shallowClassifier(nn.Module):
    def __init__(self):
        super(shallowClassifier, self).__init__()
        self.dense1 = nn.Linear(200, 100) #change to 19*200 if commenting .mean()avobe
        self.dense2 = nn.Linear(100, 10)
        
    def forward(self,x):
        x = x.view(-1, 200) #change to 200*19 if commenting .mean() above
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        
        return x
    

# Build embeddings with GTZAN time

In [2]:
# Compute and store the embeddings for each subset
if not os.path.isfile('train_embeddings.npy'):
    i=0
    train_embeddings = []
    for track in train_dataset:
        i+=1
        print('Processing track '+str(i)+' of '+str(len(train_dataset)))
        train_embeddings.append(es.TensorflowPredictMusiCNN(
            graphFilename='msd-musicnn-1.pb', output='model/dense/BiasAdd')(resample(track[0].numpy()[0])))
    train_embeddings = np.array(train_embeddings)
    np.save('train_embeddings.npy',train_embeddings)

    val_embeddings = []
    for track in val_dataset:
        val_embeddings.append(es.TensorflowPredictMusiCNN(
            graphFilename='msd-musicnn-1.pb', output='model/dense/BiasAdd')(resample(track[0].numpy()[0])))
    val_embeddings=np.array(val_embeddings)    
    np.save('val_embeddings.npy',val_embeddings)

    test_embeddings = []
    for track in test_dataset:
        test_embeddings.append(es.TensorflowPredictMusiCNN(
            graphFilename='msd-musicnn-1.pb', output='model/dense/BiasAdd')(resample(track[0].numpy()[0])))
    test_embeddings=np.array(test_embeddings)
    np.save('test_embeddings.npy',np.array(test_embeddings))
else:
    train_embeddings=np.load('train_embeddings.npy')
    val_embeddings=np.load('val_embeddings.npy')
    test_embeddings=np.load('test_embeddings.npy')

Processing track 1 of 443
Processing track 2 of 443
Processing track 3 of 443
Processing track 4 of 443
Processing track 5 of 443
Processing track 6 of 443
Processing track 7 of 443
Processing track 8 of 443
Processing track 9 of 443
Processing track 10 of 443
Processing track 11 of 443
Processing track 12 of 443
Processing track 13 of 443
Processing track 14 of 443
Processing track 15 of 443
Processing track 16 of 443
Processing track 17 of 443
Processing track 18 of 443
Processing track 19 of 443
Processing track 20 of 443
Processing track 21 of 443
Processing track 22 of 443
Processing track 23 of 443
Processing track 24 of 443
Processing track 25 of 443
Processing track 26 of 443
Processing track 27 of 443
Processing track 28 of 443
Processing track 29 of 443
Processing track 30 of 443
Processing track 31 of 443
Processing track 32 of 443
Processing track 33 of 443
Processing track 34 of 443
Processing track 35 of 443
Processing track 36 of 443
Processing track 37 of 443
Processing

Processing track 298 of 443
Processing track 299 of 443
Processing track 300 of 443
Processing track 301 of 443
Processing track 302 of 443
Processing track 303 of 443
Processing track 304 of 443
Processing track 305 of 443
Processing track 306 of 443
Processing track 307 of 443
Processing track 308 of 443
Processing track 309 of 443
Processing track 310 of 443
Processing track 311 of 443
Processing track 312 of 443
Processing track 313 of 443
Processing track 314 of 443
Processing track 315 of 443
Processing track 316 of 443
Processing track 317 of 443
Processing track 318 of 443
Processing track 319 of 443
Processing track 320 of 443
Processing track 321 of 443
Processing track 322 of 443
Processing track 323 of 443
Processing track 324 of 443
Processing track 325 of 443
Processing track 326 of 443
Processing track 327 of 443
Processing track 328 of 443
Processing track 329 of 443
Processing track 330 of 443
Processing track 331 of 443
Processing track 332 of 443
Processing track 333

In [3]:
# Embedding shapes...
train_embeddings[0].shape

(19, 200)

In [4]:
# Embedding types
train_embeddings[0].dtype

dtype('float32')

In [5]:
embtrain_dataset = Essentia_MusiCNNMSD_GTZAN_Dataset(train_dataset, train_embeddings)

# Build train loop

In [6]:
def train_test(train_embeddings, val_embeddings, test_embeddings):
    # We compute the distance of the embeddings between all songs in the training set
    emb_distance = np.zeros((len(train_embeddings), len(train_embeddings)))

    for indxA, trackA in enumerate(train_embeddings):
        for indxB, trackB in enumerate(train_embeddings):
            emb_distance[indxA, indxB] = np.linalg.norm(trackA - trackB)        

    embtrain_dataset = Essentia_MusiCNNMSD_GTZAN_Dataset(train_dataset, train_embeddings)
    train_loader = torch.utils.data.DataLoader(embtrain_dataset, batch_size=16, shuffle=True, num_workers=4)

    embval_dataset = Essentia_MusiCNNMSD_GTZAN_Dataset(val_dataset, val_embeddings)
    val_loader = torch.utils.data.DataLoader(embval_dataset, batch_size=16, shuffle=False, num_workers=4)

    embtest_dataset = Essentia_MusiCNNMSD_GTZAN_Dataset(test_dataset, test_embeddings)
    test_loader = torch.utils.data.DataLoader(embtest_dataset, batch_size=16, shuffle=False, num_workers=4)

    model = shallowClassifier()

    if torch.cuda.is_available():
        model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

    num_epochs = 2000
    train_losses = torch.zeros(num_epochs)
    val_losses = torch.zeros(num_epochs)

    bestloss = 100000.0
    for epoch in range(num_epochs):
        #The train loop
        model.train()
        for inputs, labels in train_loader:
            # Send data to the GPU
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Clear gradient and forward + loss + backward
            optimizer.zero_grad()
            outputs = model(inputs) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_losses[epoch] += loss.item()
        train_losses[epoch] /= len(train_loader)

        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Send data to the GPU
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    labels = labels.cuda()

                outputs = model(inputs) 
                loss = criterion(outputs, labels)

                val_losses[epoch] += loss.item()
            val_losses[epoch] /= len(val_loader)
            scheduler.step(val_losses[epoch])

            # If best epoch, we save parameters
            if val_losses[epoch] < bestloss :
                bestloss = val_losses[epoch]
                torch.save(model.state_dict(), 'model.pth')

        print('Epoch '+str(epoch)+': Train Loss = '+str(train_losses[epoch].item())+'. Val Loss = '+str(val_losses[epoch].item())+'.')
    print('Best validation loss :' + str(bestloss.item()))

    # Finally we compute accuracy with the test set
    model.load_state_dict(torch.load('model.pth'));
    model.eval()
    confusion_matrix = torch.zeros(len(embtrain_dataset.GTZAN_genres), len(embtrain_dataset.GTZAN_genres))
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Send data to the GPU
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    # Per-class Accuracy
    pclass_acc = confusion_matrix.diag()/confusion_matrix.sum(1)
    return torch.mean(pclass_acc).item()*100

# Train GTZAN_time

In [7]:
baseline = train_test(train_embeddings, val_embeddings, test_embeddings)

Epoch 0: Train Loss = 2.396742582321167. Val Loss = 2.302741050720215.
Epoch 1: Train Loss = 2.3672542572021484. Val Loss = 2.2801504135131836.
Epoch 2: Train Loss = 2.3425800800323486. Val Loss = 2.257930040359497.
Epoch 3: Train Loss = 2.3129024505615234. Val Loss = 2.236037015914917.
Epoch 4: Train Loss = 2.2873647212982178. Val Loss = 2.2145864963531494.
Epoch 5: Train Loss = 2.262901782989502. Val Loss = 2.1934704780578613.
Epoch 6: Train Loss = 2.240729331970215. Val Loss = 2.172711133956909.
Epoch 7: Train Loss = 2.2136614322662354. Val Loss = 2.1523585319519043.
Epoch 8: Train Loss = 2.191176652908325. Val Loss = 2.132297992706299.
Epoch 9: Train Loss = 2.1652956008911133. Val Loss = 2.1125051975250244.
Epoch 10: Train Loss = 2.1408047676086426. Val Loss = 2.0929925441741943.
Epoch 11: Train Loss = 2.118421792984009. Val Loss = 2.073817014694214.
Epoch 12: Train Loss = 2.0950369834899902. Val Loss = 2.05491042137146.
Epoch 13: Train Loss = 2.0730128288269043. Val Loss = 2.03628

Epoch 112: Train Loss = 0.974666953086853. Val Loss = 1.1177304983139038.
Epoch 113: Train Loss = 0.975354790687561. Val Loss = 1.1135919094085693.
Epoch 114: Train Loss = 0.9669046998023987. Val Loss = 1.1094673871994019.
Epoch 115: Train Loss = 0.9576854109764099. Val Loss = 1.1054285764694214.
Epoch 116: Train Loss = 0.9546635746955872. Val Loss = 1.1014955043792725.
Epoch 117: Train Loss = 0.9449706077575684. Val Loss = 1.0975805521011353.
Epoch 118: Train Loss = 0.9438691139221191. Val Loss = 1.0936866998672485.
Epoch 119: Train Loss = 0.939998984336853. Val Loss = 1.0898699760437012.
Epoch 120: Train Loss = 0.9336462616920471. Val Loss = 1.0860657691955566.
Epoch 121: Train Loss = 0.9284143447875977. Val Loss = 1.082349181175232.
Epoch 122: Train Loss = 0.9270691871643066. Val Loss = 1.0786408185958862.
Epoch 123: Train Loss = 0.919039785861969. Val Loss = 1.0749964714050293.
Epoch 124: Train Loss = 0.9209436178207397. Val Loss = 1.071426510810852.
Epoch 125: Train Loss = 0.91114

Epoch 222: Train Loss = 0.6518397927284241. Val Loss = 0.8527491092681885.
Epoch 223: Train Loss = 0.6551564931869507. Val Loss = 0.851402223110199.
Epoch 224: Train Loss = 0.6504305005073547. Val Loss = 0.8500674366950989.
Epoch 225: Train Loss = 0.6485537886619568. Val Loss = 0.8487449288368225.
Epoch 226: Train Loss = 0.6503079533576965. Val Loss = 0.8474143743515015.
Epoch 227: Train Loss = 0.6449491381645203. Val Loss = 0.8460938334465027.
Epoch 228: Train Loss = 0.6478463411331177. Val Loss = 0.8448179960250854.
Epoch 229: Train Loss = 0.642121434211731. Val Loss = 0.8435102701187134.
Epoch 230: Train Loss = 0.6405940651893616. Val Loss = 0.842229425907135.
Epoch 231: Train Loss = 0.6375018954277039. Val Loss = 0.8409919738769531.
Epoch 232: Train Loss = 0.6356317400932312. Val Loss = 0.8397502899169922.
Epoch 233: Train Loss = 0.6367629766464233. Val Loss = 0.8384296894073486.
Epoch 234: Train Loss = 0.6369333267211914. Val Loss = 0.8372380137443542.
Epoch 235: Train Loss = 0.63

Epoch 332: Train Loss = 0.5228267312049866. Val Loss = 0.753591001033783.
Epoch 333: Train Loss = 0.5232675671577454. Val Loss = 0.7530035376548767.
Epoch 334: Train Loss = 0.5239378213882446. Val Loss = 0.7524099349975586.
Epoch 335: Train Loss = 0.5221711993217468. Val Loss = 0.7518645524978638.
Epoch 336: Train Loss = 0.5224446058273315. Val Loss = 0.7513153553009033.
Epoch 337: Train Loss = 0.5207098126411438. Val Loss = 0.7507157921791077.
Epoch 338: Train Loss = 0.517365574836731. Val Loss = 0.7501510977745056.
Epoch 339: Train Loss = 0.521774172782898. Val Loss = 0.7496076822280884.
Epoch 340: Train Loss = 0.5198625326156616. Val Loss = 0.749008297920227.
Epoch 341: Train Loss = 0.517652153968811. Val Loss = 0.7484378814697266.
Epoch 342: Train Loss = 0.5178192257881165. Val Loss = 0.7478487491607666.
Epoch 343: Train Loss = 0.515666663646698. Val Loss = 0.7472935318946838.
Epoch 344: Train Loss = 0.5139620900154114. Val Loss = 0.7467749714851379.
Epoch 345: Train Loss = 0.51680

Epoch 442: Train Loss = 0.4540232717990875. Val Loss = 0.7078952789306641.
Epoch 443: Train Loss = 0.4577751159667969. Val Loss = 0.7076155543327332.
Epoch 444: Train Loss = 0.4526732861995697. Val Loss = 0.7073310017585754.
Epoch 445: Train Loss = 0.4532931447029114. Val Loss = 0.7070401906967163.
Epoch 446: Train Loss = 0.4539472758769989. Val Loss = 0.7067895531654358.
Epoch 447: Train Loss = 0.4575182795524597. Val Loss = 0.7065497636795044.
Epoch 448: Train Loss = 0.4489314556121826. Val Loss = 0.7062536478042603.
Epoch 449: Train Loss = 0.4480288326740265. Val Loss = 0.7059541940689087.
Epoch 450: Train Loss = 0.44723233580589294. Val Loss = 0.7056769728660583.
Epoch 451: Train Loss = 0.4530656337738037. Val Loss = 0.7054003477096558.
Epoch 452: Train Loss = 0.4495891034603119. Val Loss = 0.7051348090171814.
Epoch 453: Train Loss = 0.4522092640399933. Val Loss = 0.7047899961471558.
Epoch 454: Train Loss = 0.4497378468513489. Val Loss = 0.7044856548309326.
Epoch 455: Train Loss = 

...

Epoch 1528: Train Loss = 0.33357590436935425. Val Loss = 0.6668935418128967.
Epoch 1529: Train Loss = 0.3372921943664551. Val Loss = 0.6668935418128967.
Epoch 1530: Train Loss = 0.3360716700553894. Val Loss = 0.6668936014175415.
Epoch 1531: Train Loss = 0.33716946840286255. Val Loss = 0.6668935418128967.
Epoch 1532: Train Loss = 0.33567672967910767. Val Loss = 0.6668935418128967.
Epoch 1533: Train Loss = 0.3344431221485138. Val Loss = 0.6668935418128967.
Epoch 1534: Train Loss = 0.3355540633201599. Val Loss = 0.6668936014175415.
Epoch 1535: Train Loss = 0.3357401192188263. Val Loss = 0.6668936014175415.
Epoch 1536: Train Loss = 0.33601903915405273. Val Loss = 0.6668936014175415.
Epoch 1537: Train Loss = 0.3412783741950989. Val Loss = 0.6668935418128967.
Epoch 1538: Train Loss = 0.3354986310005188. Val Loss = 0.6668935418128967.
Epoch 1539: Train Loss = 0.3378537595272064. Val Loss = 0.6668935418128967.
Epoch 1540: Train Loss = 0.3357701003551483. Val Loss = 0.6668935418128967.
Epoch 15

Epoch 1636: Train Loss = 0.33983877301216125. Val Loss = 0.6668938398361206.
Epoch 1637: Train Loss = 0.33990678191185. Val Loss = 0.6668936610221863.
Epoch 1638: Train Loss = 0.33458131551742554. Val Loss = 0.6668936610221863.
Epoch 1639: Train Loss = 0.3399558365345001. Val Loss = 0.666893720626831.
Epoch 1640: Train Loss = 0.334830105304718. Val Loss = 0.666893720626831.
Epoch 1641: Train Loss = 0.3379414975643158. Val Loss = 0.666893720626831.
Epoch 1642: Train Loss = 0.341088205575943. Val Loss = 0.6668938398361206.
Epoch 1643: Train Loss = 0.3366321623325348. Val Loss = 0.6668936610221863.
Epoch 1644: Train Loss = 0.33741825819015503. Val Loss = 0.666893720626831.
Epoch 1645: Train Loss = 0.3333740234375. Val Loss = 0.666893720626831.
Epoch 1646: Train Loss = 0.33568426966667175. Val Loss = 0.666893720626831.
Epoch 1647: Train Loss = 0.3358706831932068. Val Loss = 0.666893720626831.
Epoch 1648: Train Loss = 0.33440735936164856. Val Loss = 0.6668938398361206.
Epoch 1649: Train Los

Epoch 1744: Train Loss = 0.33822154998779297. Val Loss = 0.6668939590454102.
Epoch 1745: Train Loss = 0.3351474106311798. Val Loss = 0.6668939590454102.
Epoch 1746: Train Loss = 0.3359459340572357. Val Loss = 0.6668939590454102.
Epoch 1747: Train Loss = 0.33356764912605286. Val Loss = 0.6668939590454102.
Epoch 1748: Train Loss = 0.33493438363075256. Val Loss = 0.6668939590454102.
Epoch 1749: Train Loss = 0.3358500003814697. Val Loss = 0.6668939590454102.
Epoch 1750: Train Loss = 0.3366371691226959. Val Loss = 0.6668939590454102.
Epoch 1751: Train Loss = 0.3369874060153961. Val Loss = 0.6668939590454102.
Epoch 1752: Train Loss = 0.3343186676502228. Val Loss = 0.6668939590454102.
Epoch 1753: Train Loss = 0.33475327491760254. Val Loss = 0.6668939590454102.
Epoch 1754: Train Loss = 0.3334680199623108. Val Loss = 0.6668939590454102.
Epoch 1755: Train Loss = 0.3349982798099518. Val Loss = 0.6668939590454102.
Epoch 1756: Train Loss = 0.3358888328075409. Val Loss = 0.6668939590454102.
Epoch 17

Epoch 1852: Train Loss = 0.3343214690685272. Val Loss = 0.6668940782546997.
Epoch 1853: Train Loss = 0.334047794342041. Val Loss = 0.6668940782546997.
Epoch 1854: Train Loss = 0.3342888057231903. Val Loss = 0.6668940782546997.
Epoch 1855: Train Loss = 0.3336328864097595. Val Loss = 0.6668940782546997.
Epoch 1856: Train Loss = 0.33820226788520813. Val Loss = 0.6668940782546997.
Epoch 1857: Train Loss = 0.3347001075744629. Val Loss = 0.6668940782546997.
Epoch 1858: Train Loss = 0.33339428901672363. Val Loss = 0.6668940782546997.
Epoch 1859: Train Loss = 0.33400246500968933. Val Loss = 0.6668940782546997.
Epoch 1860: Train Loss = 0.3361113667488098. Val Loss = 0.6668941974639893.
Epoch 1861: Train Loss = 0.33507058024406433. Val Loss = 0.6668941974639893.
Epoch 1862: Train Loss = 0.33876660466194153. Val Loss = 0.6668941974639893.
Epoch 1863: Train Loss = 0.33570876717567444. Val Loss = 0.6668941974639893.
Epoch 1864: Train Loss = 0.3342837393283844. Val Loss = 0.6668940186500549.
Epoch 1

Epoch 1960: Train Loss = 0.3353555500507355. Val Loss = 0.666894257068634.
Epoch 1961: Train Loss = 0.33471331000328064. Val Loss = 0.6668941974639893.
Epoch 1962: Train Loss = 0.3365192413330078. Val Loss = 0.666894257068634.
Epoch 1963: Train Loss = 0.3349149525165558. Val Loss = 0.6668941974639893.
Epoch 1964: Train Loss = 0.34079858660697937. Val Loss = 0.6668941974639893.
Epoch 1965: Train Loss = 0.33541950583457947. Val Loss = 0.6668941974639893.
Epoch 1966: Train Loss = 0.3370349109172821. Val Loss = 0.6668941974639893.
Epoch 1967: Train Loss = 0.3355548679828644. Val Loss = 0.6668941974639893.
Epoch 1968: Train Loss = 0.33793798089027405. Val Loss = 0.6668940782546997.
Epoch 1969: Train Loss = 0.3352276682853699. Val Loss = 0.6668940782546997.
Epoch 1970: Train Loss = 0.33444660902023315. Val Loss = 0.6668941974639893.
Epoch 1971: Train Loss = 0.33619245886802673. Val Loss = 0.6668941974639893.
Epoch 1972: Train Loss = 0.3383147716522217. Val Loss = 0.6668941974639893.
Epoch 19

In [8]:
print(baseline)

80.49517869949341


# Baseline is 80.50%

In [9]:
audio = resample(train_dataset[0][0].numpy()[0])
original = es.TensorflowPredictMusiCNN(graphFilename='msd-musicnn-1.pb', output='model/dense/BiasAdd')(audio)

In [10]:
original.shape

(19, 200)

In [11]:
def melspectrogram(audio):
    # Computes the mel spectrogram of audio inputs as done in the MelonPlaylist dataset
    windowing = es.Windowing(type='hann', normalized=False, zeroPadding=0)
    spectrum = es.Spectrum()
    melbands = es.MelBands(numberBands=48,
                                   sampleRate=16000,
                                   lowFrequencyBound=0,
                                   highFrequencyBound=16000/2,
                                   inputSize=(512+0)//2+1,
                                   weighting='linear',
                                   normalize='unit_tri',
                                   warpingFormula='slaneyMel',
                                   type='power')
    amp2db = es.UnaryOperator(type='lin2db', scale=2)
    result = []
    for frame in es.FrameGenerator(audio, frameSize=512, hopSize=256,
                                   startFromZero=False):
        spectrumFrame = spectrum(windowing(frame))

        melFrame = melbands(spectrumFrame)
        result.append(amp2db(melFrame))
    return np.array(result)

In [12]:
def adapt_melonInput_TensorflowPredict(melon_sample):
    db2amp = es.UnaryOperator(type='db2lin', scale=2)
    oversampled = np.zeros((len(melon_sample), melon_sample.shape[1]*2)).astype(np.float32)
    for k in range(len(melon_sample)):
        sample = np.log10(1 + (db2amp(melon_sample[k])*10000))
        oversampled[k,:]=np.interp(np.arange(96)/2, np.arange(48), sample)
    # Now we cut again, but with hop size of 93 frames as in default TensorflowPredictMusiCNN
    new = np.zeros((int(len(oversampled) / 93) - 1, 187, 96)).astype(np.float32)
    for k in range(int(len(oversampled) / 93) - 1):
        new[k]=oversampled[k*93:k*93+187]
    return np.expand_dims(new, 2)

In [13]:
modelName='msd-musicnn-1.pb'
output_layer='model/dense/BiasAdd'
input_layer='model/Placeholder'
predict = es.TensorflowPredict(graphFilename=modelName,
                               inputs=[input_layer],
                               outputs=[output_layer])


In [14]:
in_pool = Pool()
in_pool.set('model/Placeholder', adapt_melonInput_TensorflowPredict(melspectrogram(audio)))
output = predict(in_pool)
prediction = output['model/dense/BiasAdd'][:,0,0,:]

In [15]:
# TensorflowPredictMusiCNN expects mono 16kHz sample rate inputs. Resample needed
# Compute and store the embeddings for each subset
if not os.path.isfile('train_embeddings_melon.npy'):
    i=0
    train_embeddings = []
    for track in train_dataset:
        i+=1
        print('Processing track '+str(i)+' of '+str(len(train_dataset)))
        in_pool = Pool()
        in_pool.set('model/Placeholder', adapt_melonInput_TensorflowPredict(melspectrogram(resample(track[0].numpy()[0]))))
        output = predict(in_pool)
        train_embeddings.append(output['model/dense/BiasAdd'][:,0,0,:])
    train_embeddings = np.array(train_embeddings)
    np.save('train_embeddings_melon.npy', train_embeddings)
    
    val_embeddings = []
    for track in val_dataset:
        in_pool = Pool()
        in_pool.set('model/Placeholder', adapt_melonInput_TensorflowPredict(melspectrogram(resample(track[0].numpy()[0]))))
        output = predict(in_pool)
        val_embeddings.append(output['model/dense/BiasAdd'][:,0,0,:])
    val_embeddings = np.array(val_embeddings)
    np.save('val_embeddings_melon.npy', val_embeddings)
    
    test_embeddings = []
    for track in test_dataset:
        in_pool = Pool()
        in_pool.set('model/Placeholder', adapt_melonInput_TensorflowPredict(melspectrogram(resample(track[0].numpy()[0]))))
        output = predict(in_pool)
        test_embeddings.append(output['model/dense/BiasAdd'][:,0,0,:])
    test_embeddings = np.array(test_embeddings)
    np.save('test_embeddings_melon.npy', test_embeddings)    

else:
    train_embeddings=np.load('train_embeddings_melon.npy')
    val_embeddings=np.load('val_embeddings_melon.npy')
    test_embeddings=np.load('test_embeddings_melon.npy')

Processing track 1 of 443
Processing track 2 of 443
Processing track 3 of 443
Processing track 4 of 443
Processing track 5 of 443
Processing track 6 of 443
Processing track 7 of 443
Processing track 8 of 443
Processing track 9 of 443
Processing track 10 of 443
Processing track 11 of 443
Processing track 12 of 443
Processing track 13 of 443
Processing track 14 of 443
Processing track 15 of 443
Processing track 16 of 443
Processing track 17 of 443
Processing track 18 of 443
Processing track 19 of 443
Processing track 20 of 443
Processing track 21 of 443
Processing track 22 of 443
Processing track 23 of 443
Processing track 24 of 443
Processing track 25 of 443
Processing track 26 of 443
Processing track 27 of 443
Processing track 28 of 443
Processing track 29 of 443
Processing track 30 of 443
Processing track 31 of 443
Processing track 32 of 443
Processing track 33 of 443
Processing track 34 of 443
Processing track 35 of 443
Processing track 36 of 443
Processing track 37 of 443
Processing

Processing track 298 of 443
Processing track 299 of 443
Processing track 300 of 443
Processing track 301 of 443
Processing track 302 of 443
Processing track 303 of 443
Processing track 304 of 443
Processing track 305 of 443
Processing track 306 of 443
Processing track 307 of 443
Processing track 308 of 443
Processing track 309 of 443
Processing track 310 of 443
Processing track 311 of 443
Processing track 312 of 443
Processing track 313 of 443
Processing track 314 of 443
Processing track 315 of 443
Processing track 316 of 443
Processing track 317 of 443
Processing track 318 of 443
Processing track 319 of 443
Processing track 320 of 443
Processing track 321 of 443
Processing track 322 of 443
Processing track 323 of 443
Processing track 324 of 443
Processing track 325 of 443
Processing track 326 of 443
Processing track 327 of 443
Processing track 328 of 443
Processing track 329 of 443
Processing track 330 of 443
Processing track 331 of 443
Processing track 332 of 443
Processing track 333

In [16]:
# Embedding shapes...
train_embeddings[0].shape

(19, 200)

In [17]:
# Embedding types
train_embeddings[0].dtype

dtype('float32')

# Re-train, now with interpolated melspecs

In [18]:
interpolated = train_test(train_embeddings, val_embeddings, test_embeddings)

Epoch 0: Train Loss = 2.3252789974212646. Val Loss = 2.2550265789031982.
Epoch 1: Train Loss = 2.3081765174865723. Val Loss = 2.24261212348938.
Epoch 2: Train Loss = 2.2922329902648926. Val Loss = 2.230649471282959.
Epoch 3: Train Loss = 2.277916431427002. Val Loss = 2.2190253734588623.
Epoch 4: Train Loss = 2.2623863220214844. Val Loss = 2.2076776027679443.
Epoch 5: Train Loss = 2.2472281455993652. Val Loss = 2.196645975112915.
Epoch 6: Train Loss = 2.2327771186828613. Val Loss = 2.1859965324401855.
Epoch 7: Train Loss = 2.2199504375457764. Val Loss = 2.175570487976074.
Epoch 8: Train Loss = 2.2048513889312744. Val Loss = 2.1653947830200195.
Epoch 9: Train Loss = 2.1937384605407715. Val Loss = 2.1553854942321777.
Epoch 10: Train Loss = 2.1798694133758545. Val Loss = 2.1455867290496826.
Epoch 11: Train Loss = 2.1705989837646484. Val Loss = 2.1360628604888916.
Epoch 12: Train Loss = 2.1559364795684814. Val Loss = 2.1266677379608154.
Epoch 13: Train Loss = 2.1439051628112793. Val Loss = 

Epoch 112: Train Loss = 1.3773565292358398. Val Loss = 1.5359141826629639.
Epoch 113: Train Loss = 1.3748338222503662. Val Loss = 1.5320947170257568.
Epoch 114: Train Loss = 1.3693464994430542. Val Loss = 1.528309941291809.
Epoch 115: Train Loss = 1.3632160425186157. Val Loss = 1.524595022201538.
Epoch 116: Train Loss = 1.3612338304519653. Val Loss = 1.5208882093429565.
Epoch 117: Train Loss = 1.3543621301651. Val Loss = 1.517200231552124.
Epoch 118: Train Loss = 1.351680874824524. Val Loss = 1.5135796070098877.
Epoch 119: Train Loss = 1.3460792303085327. Val Loss = 1.510013461112976.
Epoch 120: Train Loss = 1.3373371362686157. Val Loss = 1.5064549446105957.
Epoch 121: Train Loss = 1.3360995054244995. Val Loss = 1.5029329061508179.
Epoch 122: Train Loss = 1.331151008605957. Val Loss = 1.499451994895935.
Epoch 123: Train Loss = 1.3250433206558228. Val Loss = 1.4959989786148071.
Epoch 124: Train Loss = 1.3227415084838867. Val Loss = 1.4924730062484741.
Epoch 125: Train Loss = 1.319013833

Epoch 223: Train Loss = 1.0119812488555908. Val Loss = 1.2549073696136475.
Epoch 224: Train Loss = 1.0044816732406616. Val Loss = 1.253200650215149.
Epoch 225: Train Loss = 0.9990993738174438. Val Loss = 1.2516758441925049.
Epoch 226: Train Loss = 0.9999211430549622. Val Loss = 1.2500649690628052.
Epoch 227: Train Loss = 0.9991627931594849. Val Loss = 1.2485358715057373.
Epoch 228: Train Loss = 0.9984008073806763. Val Loss = 1.2469818592071533.
Epoch 229: Train Loss = 0.9936846494674683. Val Loss = 1.2453848123550415.
Epoch 230: Train Loss = 0.9928448796272278. Val Loss = 1.2438740730285645.
Epoch 231: Train Loss = 0.9888457655906677. Val Loss = 1.2423838376998901.
Epoch 232: Train Loss = 0.9873232245445251. Val Loss = 1.2407596111297607.
Epoch 233: Train Loss = 0.9867410659790039. Val Loss = 1.2392780780792236.
Epoch 234: Train Loss = 0.9821511507034302. Val Loss = 1.2377440929412842.
Epoch 235: Train Loss = 0.9813895225524902. Val Loss = 1.236344575881958.
Epoch 236: Train Loss = 0.9

Epoch 333: Train Loss = 0.8292055130004883. Val Loss = 1.1288304328918457.
Epoch 334: Train Loss = 0.8268781304359436. Val Loss = 1.1280308961868286.
Epoch 335: Train Loss = 0.8264961838722229. Val Loss = 1.1272021532058716.
Epoch 336: Train Loss = 0.8239536285400391. Val Loss = 1.1264264583587646.
Epoch 337: Train Loss = 0.8214799761772156. Val Loss = 1.1255450248718262.
Epoch 338: Train Loss = 0.822638213634491. Val Loss = 1.1248255968093872.
Epoch 339: Train Loss = 0.8181975483894348. Val Loss = 1.1240884065628052.
Epoch 340: Train Loss = 0.8199825286865234. Val Loss = 1.1232774257659912.
Epoch 341: Train Loss = 0.8188206553459167. Val Loss = 1.1224136352539062.
Epoch 342: Train Loss = 0.8185538649559021. Val Loss = 1.121606707572937.
Epoch 343: Train Loss = 0.8145328164100647. Val Loss = 1.1208164691925049.
Epoch 344: Train Loss = 0.8138689994812012. Val Loss = 1.1200175285339355.
Epoch 345: Train Loss = 0.8123237490653992. Val Loss = 1.1192455291748047.
Epoch 346: Train Loss = 0.8

Epoch 443: Train Loss = 0.7247124910354614. Val Loss = 1.0607190132141113.
Epoch 444: Train Loss = 0.7248898148536682. Val Loss = 1.0603058338165283.
Epoch 445: Train Loss = 0.7192303538322449. Val Loss = 1.0598653554916382.
Epoch 446: Train Loss = 0.7199193239212036. Val Loss = 1.059506893157959.
Epoch 447: Train Loss = 0.7165449857711792. Val Loss = 1.059133768081665.
Epoch 448: Train Loss = 0.7196346521377563. Val Loss = 1.0586628913879395.
Epoch 449: Train Loss = 0.7206173539161682. Val Loss = 1.0581495761871338.
Epoch 450: Train Loss = 0.7159423232078552. Val Loss = 1.0577223300933838.
Epoch 451: Train Loss = 0.7182666063308716. Val Loss = 1.0571510791778564.
Epoch 452: Train Loss = 0.7166227698326111. Val Loss = 1.0566521883010864.
Epoch 453: Train Loss = 0.7175742387771606. Val Loss = 1.0563104152679443.
Epoch 454: Train Loss = 0.7123107314109802. Val Loss = 1.0558981895446777.
Epoch 455: Train Loss = 0.7143031358718872. Val Loss = 1.0553672313690186.
Epoch 456: Train Loss = 0.7

...

Epoch 1533: Train Loss = 0.4193399250507355. Val Loss = 0.9180306196212769.
Epoch 1534: Train Loss = 0.42142361402511597. Val Loss = 0.9180008769035339.
Epoch 1535: Train Loss = 0.42109984159469604. Val Loss = 0.9180014729499817.
Epoch 1536: Train Loss = 0.4219896197319031. Val Loss = 0.9179942011833191.
Epoch 1537: Train Loss = 0.41875243186950684. Val Loss = 0.9179763197898865.
Epoch 1538: Train Loss = 0.424046128988266. Val Loss = 0.917959451675415.
Epoch 1539: Train Loss = 0.4268798828125. Val Loss = 0.917960524559021.
Epoch 1540: Train Loss = 0.4236254096031189. Val Loss = 0.9179598093032837.
Epoch 1541: Train Loss = 0.41908153891563416. Val Loss = 0.9179537892341614.
Epoch 1542: Train Loss = 0.4187963902950287. Val Loss = 0.9179483652114868.
Epoch 1543: Train Loss = 0.4261274039745331. Val Loss = 0.9179510474205017.
Epoch 1544: Train Loss = 0.4182327091693878. Val Loss = 0.9179463982582092.
Epoch 1545: Train Loss = 0.4234616160392761. Val Loss = 0.9179282784461975.
Epoch 1546: Tr

Epoch 1641: Train Loss = 0.41950228810310364. Val Loss = 0.9179248213768005.
Epoch 1642: Train Loss = 0.41801542043685913. Val Loss = 0.9179248213768005.
Epoch 1643: Train Loss = 0.4217877686023712. Val Loss = 0.9179248213768005.
Epoch 1644: Train Loss = 0.4216398298740387. Val Loss = 0.9179248213768005.
Epoch 1645: Train Loss = 0.4216762185096741. Val Loss = 0.9179248809814453.
Epoch 1646: Train Loss = 0.4191600978374481. Val Loss = 0.9179248809814453.
Epoch 1647: Train Loss = 0.41960078477859497. Val Loss = 0.9179247617721558.
Epoch 1648: Train Loss = 0.4201766550540924. Val Loss = 0.9179248809814453.
Epoch 1649: Train Loss = 0.42028024792671204. Val Loss = 0.9179247617721558.
Epoch 1650: Train Loss = 0.4208875298500061. Val Loss = 0.9179248213768005.
Epoch 1651: Train Loss = 0.42164140939712524. Val Loss = 0.9179248213768005.
Epoch 1652: Train Loss = 0.41922858357429504. Val Loss = 0.9179248809814453.
Epoch 1653: Train Loss = 0.4233629107475281. Val Loss = 0.9179248809814453.
Epoch 

Epoch 1749: Train Loss = 0.4192357361316681. Val Loss = 0.9179247617721558.
Epoch 1750: Train Loss = 0.41882967948913574. Val Loss = 0.9179247617721558.
Epoch 1751: Train Loss = 0.41843268275260925. Val Loss = 0.9179248213768005.
Epoch 1752: Train Loss = 0.42057672142982483. Val Loss = 0.9179247617721558.
Epoch 1753: Train Loss = 0.42383313179016113. Val Loss = 0.9179247617721558.
Epoch 1754: Train Loss = 0.42455074191093445. Val Loss = 0.9179247617721558.
Epoch 1755: Train Loss = 0.42459163069725037. Val Loss = 0.9179247617721558.
Epoch 1756: Train Loss = 0.41999581456184387. Val Loss = 0.9179247617721558.
Epoch 1757: Train Loss = 0.41989877820014954. Val Loss = 0.9179247617721558.
Epoch 1758: Train Loss = 0.4225218892097473. Val Loss = 0.9179247617721558.
Epoch 1759: Train Loss = 0.4192938804626465. Val Loss = 0.9179247617721558.
Epoch 1760: Train Loss = 0.4220573306083679. Val Loss = 0.9179247617721558.
Epoch 1761: Train Loss = 0.41725704073905945. Val Loss = 0.9179248213768005.
Epo

Epoch 1857: Train Loss = 0.41757720708847046. Val Loss = 0.9179250001907349.
Epoch 1858: Train Loss = 0.4214913845062256. Val Loss = 0.9179250001907349.
Epoch 1859: Train Loss = 0.422050803899765. Val Loss = 0.9179250001907349.
Epoch 1860: Train Loss = 0.4194900095462799. Val Loss = 0.9179248809814453.
Epoch 1861: Train Loss = 0.4189964830875397. Val Loss = 0.9179249405860901.
Epoch 1862: Train Loss = 0.419196218252182. Val Loss = 0.9179249405860901.
Epoch 1863: Train Loss = 0.42257076501846313. Val Loss = 0.9179249405860901.
Epoch 1864: Train Loss = 0.42629432678222656. Val Loss = 0.9179250001907349.
Epoch 1865: Train Loss = 0.4225451350212097. Val Loss = 0.9179249405860901.
Epoch 1866: Train Loss = 0.41991931200027466. Val Loss = 0.9179249405860901.
Epoch 1867: Train Loss = 0.4268219470977783. Val Loss = 0.9179248809814453.
Epoch 1868: Train Loss = 0.41859322786331177. Val Loss = 0.9179248809814453.
Epoch 1869: Train Loss = 0.4202413856983185. Val Loss = 0.9179248213768005.
Epoch 187

Epoch 1965: Train Loss = 0.4217360019683838. Val Loss = 0.9179250001907349.
Epoch 1966: Train Loss = 0.42023688554763794. Val Loss = 0.9179250001907349.
Epoch 1967: Train Loss = 0.42188408970832825. Val Loss = 0.9179250001907349.
Epoch 1968: Train Loss = 0.42352721095085144. Val Loss = 0.9179251194000244.
Epoch 1969: Train Loss = 0.41939741373062134. Val Loss = 0.9179251194000244.
Epoch 1970: Train Loss = 0.42157092690467834. Val Loss = 0.9179251194000244.
Epoch 1971: Train Loss = 0.4196556508541107. Val Loss = 0.9179251194000244.
Epoch 1972: Train Loss = 0.41831448674201965. Val Loss = 0.9179251194000244.
Epoch 1973: Train Loss = 0.41892385482788086. Val Loss = 0.9179249405860901.
Epoch 1974: Train Loss = 0.42036542296409607. Val Loss = 0.9179249405860901.
Epoch 1975: Train Loss = 0.4219522476196289. Val Loss = 0.9179249405860901.
Epoch 1976: Train Loss = 0.41801002621650696. Val Loss = 0.9179249405860901.
Epoch 1977: Train Loss = 0.41945981979370117. Val Loss = 0.9179249405860901.
Ep

In [19]:
print(interpolated)

74.57183599472046


# Now with random embeddings

In [20]:
np.random.seed(42)

In [21]:
rand = train_test(np.reshape(5*np.random.randn(train_embeddings.size).astype(np.float32), train_embeddings.shape), 
                 np.reshape(5*np.random.randn(val_embeddings.size).astype(np.float32), val_embeddings.shape), 
                 np.reshape(5*np.random.randn(test_embeddings.size).astype(np.float32), test_embeddings.shape))

Epoch 0: Train Loss = 2.32436203956604. Val Loss = 2.3439722061157227.
Epoch 1: Train Loss = 2.323726177215576. Val Loss = 2.3439137935638428.
Epoch 2: Train Loss = 2.3235666751861572. Val Loss = 2.3438515663146973.
Epoch 3: Train Loss = 2.323732852935791. Val Loss = 2.3437914848327637.
Epoch 4: Train Loss = 2.323045015335083. Val Loss = 2.343726873397827.
Epoch 5: Train Loss = 2.3216471672058105. Val Loss = 2.3436672687530518.
Epoch 6: Train Loss = 2.3228304386138916. Val Loss = 2.3436036109924316.
Epoch 7: Train Loss = 2.321195602416992. Val Loss = 2.3435475826263428.
Epoch 8: Train Loss = 2.3201773166656494. Val Loss = 2.343485116958618.
Epoch 9: Train Loss = 2.320592164993286. Val Loss = 2.343426465988159.
Epoch 10: Train Loss = 2.320301055908203. Val Loss = 2.3433682918548584.
Epoch 11: Train Loss = 2.319483518600464. Val Loss = 2.3433098793029785.
Epoch 12: Train Loss = 2.31825852394104. Val Loss = 2.343247652053833.
Epoch 13: Train Loss = 2.3187406063079834. Val Loss = 2.3431901

Epoch 113: Train Loss = 2.280517578125. Val Loss = 2.3389296531677246.
Epoch 114: Train Loss = 2.2796449661254883. Val Loss = 2.3388986587524414.
Epoch 115: Train Loss = 2.2796146869659424. Val Loss = 2.3388731479644775.
Epoch 116: Train Loss = 2.2798893451690674. Val Loss = 2.338844060897827.
Epoch 117: Train Loss = 2.278202772140503. Val Loss = 2.3388164043426514.
Epoch 118: Train Loss = 2.2793688774108887. Val Loss = 2.3387858867645264.
Epoch 119: Train Loss = 2.278088331222534. Val Loss = 2.3387582302093506.
Epoch 120: Train Loss = 2.278210163116455. Val Loss = 2.338726758956909.
Epoch 121: Train Loss = 2.276932954788208. Val Loss = 2.338695526123047.
Epoch 122: Train Loss = 2.277667760848999. Val Loss = 2.3386666774749756.
Epoch 123: Train Loss = 2.27669095993042. Val Loss = 2.3386406898498535.
Epoch 124: Train Loss = 2.2753407955169678. Val Loss = 2.3386130332946777.
Epoch 125: Train Loss = 2.2767162322998047. Val Loss = 2.3385872840881348.
Epoch 126: Train Loss = 2.2755186557769

Epoch 224: Train Loss = 2.259437322616577. Val Loss = 2.3376495838165283.
Epoch 225: Train Loss = 2.2610604763031006. Val Loss = 2.3376495838165283.
Epoch 226: Train Loss = 2.260190010070801. Val Loss = 2.3376495838165283.
Epoch 227: Train Loss = 2.2607064247131348. Val Loss = 2.3376495838165283.
Epoch 228: Train Loss = 2.2606148719787598. Val Loss = 2.3376495838165283.
Epoch 229: Train Loss = 2.2606446743011475. Val Loss = 2.3376495838165283.
Epoch 230: Train Loss = 2.2603654861450195. Val Loss = 2.3376495838165283.
Epoch 231: Train Loss = 2.2598845958709717. Val Loss = 2.3376495838165283.
Epoch 232: Train Loss = 2.2599358558654785. Val Loss = 2.3376495838165283.
Epoch 233: Train Loss = 2.2604758739471436. Val Loss = 2.337649345397949.
Epoch 234: Train Loss = 2.2598612308502197. Val Loss = 2.337649345397949.
Epoch 235: Train Loss = 2.2598073482513428. Val Loss = 2.337649345397949.
Epoch 236: Train Loss = 2.2596890926361084. Val Loss = 2.337649345397949.
Epoch 237: Train Loss = 2.25819

Epoch 336: Train Loss = 2.261859893798828. Val Loss = 2.337649345397949.
Epoch 337: Train Loss = 2.259434938430786. Val Loss = 2.3376495838165283.
Epoch 338: Train Loss = 2.2597362995147705. Val Loss = 2.3376495838165283.
Epoch 339: Train Loss = 2.2591538429260254. Val Loss = 2.3376495838165283.
Epoch 340: Train Loss = 2.2600321769714355. Val Loss = 2.3376495838165283.
Epoch 341: Train Loss = 2.2602200508117676. Val Loss = 2.3376495838165283.
Epoch 342: Train Loss = 2.260223627090454. Val Loss = 2.337649345397949.
Epoch 343: Train Loss = 2.259397029876709. Val Loss = 2.3376495838165283.
Epoch 344: Train Loss = 2.2600865364074707. Val Loss = 2.3376495838165283.
Epoch 345: Train Loss = 2.2610833644866943. Val Loss = 2.3376495838165283.
Epoch 346: Train Loss = 2.2605748176574707. Val Loss = 2.3376495838165283.
Epoch 347: Train Loss = 2.261261463165283. Val Loss = 2.3376495838165283.
Epoch 348: Train Loss = 2.259505271911621. Val Loss = 2.3376495838165283.
Epoch 349: Train Loss = 2.2607955

Epoch 447: Train Loss = 2.2604615688323975. Val Loss = 2.3376495838165283.
Epoch 448: Train Loss = 2.260007381439209. Val Loss = 2.3376495838165283.
Epoch 449: Train Loss = 2.260596752166748. Val Loss = 2.3376495838165283.
Epoch 450: Train Loss = 2.2599117755889893. Val Loss = 2.3376498222351074.
Epoch 451: Train Loss = 2.260814666748047. Val Loss = 2.3376498222351074.
Epoch 452: Train Loss = 2.2606377601623535. Val Loss = 2.3376498222351074.
Epoch 453: Train Loss = 2.2606756687164307. Val Loss = 2.3376498222351074.
Epoch 454: Train Loss = 2.2600693702697754. Val Loss = 2.3376498222351074.
Epoch 455: Train Loss = 2.2597572803497314. Val Loss = 2.3376498222351074.
Epoch 456: Train Loss = 2.2583627700805664. Val Loss = 2.3376498222351074.
Epoch 457: Train Loss = 2.2612218856811523. Val Loss = 2.3376498222351074.
Epoch 458: Train Loss = 2.259697675704956. Val Loss = 2.3376498222351074.
Epoch 459: Train Loss = 2.259103775024414. Val Loss = 2.3376498222351074.
Epoch 460: Train Loss = 2.2599

...

Epoch 1451: Train Loss = 2.2614734172821045. Val Loss = 2.337648868560791.
Epoch 1452: Train Loss = 2.26149845123291. Val Loss = 2.337648868560791.
Epoch 1453: Train Loss = 2.259819984436035. Val Loss = 2.337648868560791.
Epoch 1454: Train Loss = 2.2607405185699463. Val Loss = 2.337648868560791.
Epoch 1455: Train Loss = 2.2595694065093994. Val Loss = 2.337648868560791.
Epoch 1456: Train Loss = 2.257981300354004. Val Loss = 2.337648868560791.
Epoch 1457: Train Loss = 2.258849620819092. Val Loss = 2.337648868560791.
Epoch 1458: Train Loss = 2.259713888168335. Val Loss = 2.33764910697937.
Epoch 1459: Train Loss = 2.261051654815674. Val Loss = 2.337648868560791.
Epoch 1460: Train Loss = 2.259258508682251. Val Loss = 2.33764910697937.
Epoch 1461: Train Loss = 2.2614901065826416. Val Loss = 2.33764910697937.
Epoch 1462: Train Loss = 2.2601611614227295. Val Loss = 2.33764910697937.
Epoch 1463: Train Loss = 2.261193037033081. Val Loss = 2.33764910697937.
Epoch 1464: Train Loss = 2.259458065032

Epoch 1562: Train Loss = 2.260854721069336. Val Loss = 2.33764910697937.
Epoch 1563: Train Loss = 2.260094165802002. Val Loss = 2.33764910697937.
Epoch 1564: Train Loss = 2.2590880393981934. Val Loss = 2.33764910697937.
Epoch 1565: Train Loss = 2.260345220565796. Val Loss = 2.33764910697937.
Epoch 1566: Train Loss = 2.259732484817505. Val Loss = 2.33764910697937.
Epoch 1567: Train Loss = 2.2613606452941895. Val Loss = 2.33764910697937.
Epoch 1568: Train Loss = 2.259793996810913. Val Loss = 2.33764910697937.
Epoch 1569: Train Loss = 2.260828733444214. Val Loss = 2.33764910697937.
Epoch 1570: Train Loss = 2.2610769271850586. Val Loss = 2.33764910697937.
Epoch 1571: Train Loss = 2.259807586669922. Val Loss = 2.33764910697937.
Epoch 1572: Train Loss = 2.2604000568389893. Val Loss = 2.33764910697937.
Epoch 1573: Train Loss = 2.2607576847076416. Val Loss = 2.33764910697937.
Epoch 1574: Train Loss = 2.259523630142212. Val Loss = 2.33764910697937.
Epoch 1575: Train Loss = 2.2606265544891357. V

Epoch 1674: Train Loss = 2.260512590408325. Val Loss = 2.33764910697937.
Epoch 1675: Train Loss = 2.2603132724761963. Val Loss = 2.33764910697937.
Epoch 1676: Train Loss = 2.2597644329071045. Val Loss = 2.33764910697937.
Epoch 1677: Train Loss = 2.2614591121673584. Val Loss = 2.33764910697937.
Epoch 1678: Train Loss = 2.2601001262664795. Val Loss = 2.33764910697937.
Epoch 1679: Train Loss = 2.2606914043426514. Val Loss = 2.33764910697937.
Epoch 1680: Train Loss = 2.2589685916900635. Val Loss = 2.33764910697937.
Epoch 1681: Train Loss = 2.260385274887085. Val Loss = 2.33764910697937.
Epoch 1682: Train Loss = 2.25932240486145. Val Loss = 2.33764910697937.
Epoch 1683: Train Loss = 2.2608482837677. Val Loss = 2.33764910697937.
Epoch 1684: Train Loss = 2.260205030441284. Val Loss = 2.33764910697937.
Epoch 1685: Train Loss = 2.2606027126312256. Val Loss = 2.33764910697937.
Epoch 1686: Train Loss = 2.2598888874053955. Val Loss = 2.33764910697937.
Epoch 1687: Train Loss = 2.2598514556884766. V

Epoch 1786: Train Loss = 2.260957956314087. Val Loss = 2.33764910697937.
Epoch 1787: Train Loss = 2.2601311206817627. Val Loss = 2.33764910697937.
Epoch 1788: Train Loss = 2.2608563899993896. Val Loss = 2.33764910697937.
Epoch 1789: Train Loss = 2.259551763534546. Val Loss = 2.33764910697937.
Epoch 1790: Train Loss = 2.2607548236846924. Val Loss = 2.33764910697937.
Epoch 1791: Train Loss = 2.2598395347595215. Val Loss = 2.33764910697937.
Epoch 1792: Train Loss = 2.260133981704712. Val Loss = 2.33764910697937.
Epoch 1793: Train Loss = 2.2610890865325928. Val Loss = 2.33764910697937.
Epoch 1794: Train Loss = 2.260904312133789. Val Loss = 2.33764910697937.
Epoch 1795: Train Loss = 2.2591450214385986. Val Loss = 2.33764910697937.
Epoch 1796: Train Loss = 2.2596373558044434. Val Loss = 2.33764910697937.
Epoch 1797: Train Loss = 2.260707378387451. Val Loss = 2.33764910697937.
Epoch 1798: Train Loss = 2.2605271339416504. Val Loss = 2.33764910697937.
Epoch 1799: Train Loss = 2.2603964805603027

Epoch 1897: Train Loss = 2.2593815326690674. Val Loss = 2.337648868560791.
Epoch 1898: Train Loss = 2.260730266571045. Val Loss = 2.337648868560791.
Epoch 1899: Train Loss = 2.259798288345337. Val Loss = 2.337648868560791.
Epoch 1900: Train Loss = 2.2605855464935303. Val Loss = 2.337648868560791.
Epoch 1901: Train Loss = 2.2597947120666504. Val Loss = 2.337648868560791.
Epoch 1902: Train Loss = 2.2606582641601562. Val Loss = 2.337648868560791.
Epoch 1903: Train Loss = 2.2603325843811035. Val Loss = 2.337648868560791.
Epoch 1904: Train Loss = 2.2607405185699463. Val Loss = 2.337648868560791.
Epoch 1905: Train Loss = 2.2602646350860596. Val Loss = 2.337648868560791.
Epoch 1906: Train Loss = 2.260333299636841. Val Loss = 2.337648868560791.
Epoch 1907: Train Loss = 2.259042978286743. Val Loss = 2.337648868560791.
Epoch 1908: Train Loss = 2.259925365447998. Val Loss = 2.337648868560791.
Epoch 1909: Train Loss = 2.2587685585021973. Val Loss = 2.337648868560791.
Epoch 1910: Train Loss = 2.259

In [22]:
print(rand)

9.379075467586517


# Now with random musiCNN

In [23]:
class Conv_V(nn.Module):
    # vertical convolution
    def __init__(self, input_channels, output_channels, filter_shape):
        super(Conv_V, self).__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, filter_shape,
                              padding=(0, filter_shape[1]//2))
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        freq = x.size(2)
        out = nn.MaxPool2d((freq, 1), stride=(freq, 1))(x)
        out = out.squeeze(2)
        return out


class Conv_H(nn.Module):
    # horizontal convolution
    def __init__(self, input_channels, output_channels, filter_length):
        super(Conv_H, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, filter_length,
                              padding=filter_length//2)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        freq = x.size(2)
        out = nn.AvgPool2d((freq, 1), stride=(freq, 1))(x)
        out = out.squeeze(2)
        out = self.relu(self.bn(self.conv(out)))
        return out

class Conv_1d(nn.Module):
    def __init__(self, input_channels, output_channels, shape=3, stride=1, pooling=2):
        super(Conv_1d, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, shape, stride=stride, padding=shape//2)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool1d(pooling)
    def forward(self, x):
        out = self.mp(self.relu(self.bn(self.conv(x))))
        return out
    
class Musicnn(nn.Module):
    '''
    Pons et al. 2017
    End-to-end learning for music audio tagging at scale.
    This is the updated implementation of the original paper. Referred to the Musicnn code.
    https://github.com/jordipons/musicnn
    '''
    def __init__(self,
                sample_rate=16000,
                n_fft=512,
                f_min=0.0,
                f_max=8000.0,
                n_mels=96,
                n_class=50,
                dataset='mtat'):
        super(Musicnn, self).__init__()

        # Spectrogram
        self.spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate,
                                                         n_fft=n_fft,
                                                         f_min=f_min,
                                                         f_max=f_max,
                                                         n_mels=n_mels)
        self.to_db = torchaudio.transforms.AmplitudeToDB()
        self.spec_bn = nn.BatchNorm2d(1)

        # Pons front-end
        m1 = Conv_V(1, 204, (int(0.7*96), 7))
        m2 = Conv_V(1, 204, (int(0.4*96), 7))
        m3 = Conv_H(1, 51, 129)
        m4 = Conv_H(1, 51, 65)
        m5 = Conv_H(1, 51, 33)
        self.layers = nn.ModuleList([m1, m2, m3, m4, m5])

        # Pons back-end
        backend_channel= 512 if dataset=='msd' else 64
        self.layer1 = Conv_1d(561, backend_channel, 7, 1, 1)
        self.layer2 = Conv_1d(backend_channel, backend_channel, 7, 1, 1)
        self.layer3 = Conv_1d(backend_channel, backend_channel, 7, 1, 1)

        # Dense
        dense_channel = 500 if dataset=='msd' else 200
        self.dense1 = nn.Linear((561+(backend_channel*3))*2, dense_channel)
        self.bn = nn.BatchNorm1d(dense_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.dense2 = nn.Linear(dense_channel, n_class)

    def forward(self, x):
        # Spectrogram
        x = self.spec(x)
        x = self.to_db(x)
        x = x.unsqueeze(1)
        x = self.spec_bn(x)

        # Pons front-end
        out = []
        for layer in self.layers:
            out.append(layer(x))
        out = torch.cat(out, dim=1)

        # Pons back-end
        length = out.size(2)
        res1 = self.layer1(out)
        res2 = self.layer2(res1) + res1
        res3 = self.layer3(res2) + res2
        out = torch.cat([out, res1, res2, res3], 1)

        mp = nn.MaxPool1d(length)(out)
        avgp = nn.AvgPool1d(length)(out)
        out = torch.cat([mp, avgp], dim=1)
        out = out.squeeze(2)

        out = self.relu(self.bn(self.dense1(out)))
        out = self.dropout(out)
        #out = self.dense2(out)
        #out = nn.Sigmoid()(out)

        return out
rand_musiCNN = Musicnn()
rand_musiCNN.eval();

In [24]:
# Compute and store the embeddings for each subset
if not os.path.isfile('train_embeddings_random.npy'):
    i=0
    train_embeddings = []
    for track in train_dataset:
        i+=1
        print('Processing track '+str(i)+' of '+str(len(train_dataset)))
        train_embeddings.append(rand_musiCNN(torch.from_numpy(resample(track[0].numpy()[0])).unsqueeze(0)).detach().numpy())
    train_embeddings = np.array(train_embeddings)
    np.save('train_embeddings_random.npy', train_embeddings)
    
    val_embeddings = []
    for track in val_dataset:
        val_embeddings.append(rand_musiCNN(torch.from_numpy(resample(track[0].numpy()[0])).unsqueeze(0)).detach().numpy())
    val_embeddings = np.array(val_embeddings)
    np.save('val_embeddings_random.npy', val_embeddings)
    
    test_embeddings = []
    for track in test_dataset:
        test_embeddings.append(rand_musiCNN(torch.from_numpy(resample(track[0].numpy()[0])).unsqueeze(0)).detach().numpy())
    test_embeddings = np.array(test_embeddings)
    np.save('test_embeddings_random.npy', test_embeddings)    

else:
    train_embeddings=np.load('train_embeddings_random.npy')
    val_embeddings=np.load('val_embeddings_random.npy')
    test_embeddings=np.load('test_embeddings_random.npy')

Processing track 1 of 443
Processing track 2 of 443
Processing track 3 of 443
Processing track 4 of 443
Processing track 5 of 443
Processing track 6 of 443
Processing track 7 of 443
Processing track 8 of 443
Processing track 9 of 443
Processing track 10 of 443
Processing track 11 of 443
Processing track 12 of 443
Processing track 13 of 443
Processing track 14 of 443
Processing track 15 of 443
Processing track 16 of 443
Processing track 17 of 443
Processing track 18 of 443
Processing track 19 of 443
Processing track 20 of 443
Processing track 21 of 443
Processing track 22 of 443
Processing track 23 of 443
Processing track 24 of 443
Processing track 25 of 443
Processing track 26 of 443
Processing track 27 of 443
Processing track 28 of 443
Processing track 29 of 443
Processing track 30 of 443
Processing track 31 of 443
Processing track 32 of 443
Processing track 33 of 443
Processing track 34 of 443
Processing track 35 of 443
Processing track 36 of 443
Processing track 37 of 443
Processing

Processing track 298 of 443
Processing track 299 of 443
Processing track 300 of 443
Processing track 301 of 443
Processing track 302 of 443
Processing track 303 of 443
Processing track 304 of 443
Processing track 305 of 443
Processing track 306 of 443
Processing track 307 of 443
Processing track 308 of 443
Processing track 309 of 443
Processing track 310 of 443
Processing track 311 of 443
Processing track 312 of 443
Processing track 313 of 443
Processing track 314 of 443
Processing track 315 of 443
Processing track 316 of 443
Processing track 317 of 443
Processing track 318 of 443
Processing track 319 of 443
Processing track 320 of 443
Processing track 321 of 443
Processing track 322 of 443
Processing track 323 of 443
Processing track 324 of 443
Processing track 325 of 443
Processing track 326 of 443
Processing track 327 of 443
Processing track 328 of 443
Processing track 329 of 443
Processing track 330 of 443
Processing track 331 of 443
Processing track 332 of 443
Processing track 333

In [25]:
random_net = train_test(train_embeddings, val_embeddings, test_embeddings)

Epoch 0: Train Loss = 3.1070022583007812. Val Loss = 2.6462361812591553.
Epoch 1: Train Loss = 2.743884563446045. Val Loss = 2.4482369422912598.
Epoch 2: Train Loss = 2.573045253753662. Val Loss = 2.3497350215911865.
Epoch 3: Train Loss = 2.4660449028015137. Val Loss = 2.2921526432037354.
Epoch 4: Train Loss = 2.403522253036499. Val Loss = 2.2526986598968506.
Epoch 5: Train Loss = 2.3515665531158447. Val Loss = 2.2249488830566406.
Epoch 6: Train Loss = 2.316086530685425. Val Loss = 2.2037503719329834.
Epoch 7: Train Loss = 2.292104959487915. Val Loss = 2.1874403953552246.
Epoch 8: Train Loss = 2.2674314975738525. Val Loss = 2.1744139194488525.
Epoch 9: Train Loss = 2.253220796585083. Val Loss = 2.1644346714019775.
Epoch 10: Train Loss = 2.235412120819092. Val Loss = 2.157454490661621.
Epoch 11: Train Loss = 2.2214887142181396. Val Loss = 2.1503429412841797.
Epoch 12: Train Loss = 2.2066733837127686. Val Loss = 2.1446447372436523.
Epoch 13: Train Loss = 2.195754289627075. Val Loss = 2.1

Epoch 112: Train Loss = 1.8638875484466553. Val Loss = 2.010989189147949.
Epoch 113: Train Loss = 1.8649464845657349. Val Loss = 2.0106558799743652.
Epoch 114: Train Loss = 1.8548732995986938. Val Loss = 2.0101566314697266.
Epoch 115: Train Loss = 1.8606854677200317. Val Loss = 2.0098161697387695.
Epoch 116: Train Loss = 1.8608429431915283. Val Loss = 2.007805824279785.
Epoch 117: Train Loss = 1.8606959581375122. Val Loss = 2.0082647800445557.
Epoch 118: Train Loss = 1.8606383800506592. Val Loss = 2.006816864013672.
Epoch 119: Train Loss = 1.8568893671035767. Val Loss = 2.0063321590423584.
Epoch 120: Train Loss = 1.8567739725112915. Val Loss = 2.0060529708862305.
Epoch 121: Train Loss = 1.8533551692962646. Val Loss = 2.006051540374756.
Epoch 122: Train Loss = 1.8524178266525269. Val Loss = 2.0054545402526855.
Epoch 123: Train Loss = 1.847302794456482. Val Loss = 2.004030227661133.
Epoch 124: Train Loss = 1.8473254442214966. Val Loss = 2.003417730331421.
Epoch 125: Train Loss = 1.847398

Epoch 223: Train Loss = 1.736012578010559. Val Loss = 1.9581488370895386.
Epoch 224: Train Loss = 1.7330116033554077. Val Loss = 1.958638310432434.
Epoch 225: Train Loss = 1.7309303283691406. Val Loss = 1.9576722383499146.
Epoch 226: Train Loss = 1.7343394756317139. Val Loss = 1.9576503038406372.
Epoch 227: Train Loss = 1.7329447269439697. Val Loss = 1.956828236579895.
Epoch 228: Train Loss = 1.7286632061004639. Val Loss = 1.9572256803512573.
Epoch 229: Train Loss = 1.7320518493652344. Val Loss = 1.9562371969223022.
Epoch 230: Train Loss = 1.7287485599517822. Val Loss = 1.9557310342788696.
Epoch 231: Train Loss = 1.7288824319839478. Val Loss = 1.9548283815383911.
Epoch 232: Train Loss = 1.725329041481018. Val Loss = 1.9556936025619507.
Epoch 233: Train Loss = 1.7202930450439453. Val Loss = 1.955368161201477.
Epoch 234: Train Loss = 1.727330207824707. Val Loss = 1.9543076753616333.
Epoch 235: Train Loss = 1.7252815961837769. Val Loss = 1.9546793699264526.
Epoch 236: Train Loss = 1.72664

Epoch 333: Train Loss = 1.6444942951202393. Val Loss = 1.9278552532196045.
Epoch 334: Train Loss = 1.6475456953048706. Val Loss = 1.9275542497634888.
Epoch 335: Train Loss = 1.6449601650238037. Val Loss = 1.927337408065796.
Epoch 336: Train Loss = 1.6427654027938843. Val Loss = 1.9273308515548706.
Epoch 337: Train Loss = 1.64158034324646. Val Loss = 1.9268501996994019.
Epoch 338: Train Loss = 1.6379793882369995. Val Loss = 1.9260954856872559.
Epoch 339: Train Loss = 1.6384961605072021. Val Loss = 1.926649570465088.
Epoch 340: Train Loss = 1.643657922744751. Val Loss = 1.9255974292755127.
Epoch 341: Train Loss = 1.6410229206085205. Val Loss = 1.924140453338623.
Epoch 342: Train Loss = 1.6371514797210693. Val Loss = 1.9239579439163208.
Epoch 343: Train Loss = 1.6400240659713745. Val Loss = 1.92355477809906.
Epoch 344: Train Loss = 1.638562560081482. Val Loss = 1.9239475727081299.
Epoch 345: Train Loss = 1.6317815780639648. Val Loss = 1.9235286712646484.
Epoch 346: Train Loss = 1.63476490

Epoch 444: Train Loss = 1.5739572048187256. Val Loss = 1.9012631177902222.
Epoch 445: Train Loss = 1.5703012943267822. Val Loss = 1.9010887145996094.
Epoch 446: Train Loss = 1.5697778463363647. Val Loss = 1.9010034799575806.
Epoch 447: Train Loss = 1.5669152736663818. Val Loss = 1.901320219039917.
Epoch 448: Train Loss = 1.5660635232925415. Val Loss = 1.9010202884674072.
Epoch 449: Train Loss = 1.5681018829345703. Val Loss = 1.8993312120437622.
Epoch 450: Train Loss = 1.5678802728652954. Val Loss = 1.8997111320495605.
Epoch 451: Train Loss = 1.5664678812026978. Val Loss = 1.8978962898254395.
Epoch 452: Train Loss = 1.5658848285675049. Val Loss = 1.8988392353057861.
Epoch 453: Train Loss = 1.5697963237762451. Val Loss = 1.9004342555999756.
Epoch 454: Train Loss = 1.5622092485427856. Val Loss = 1.9009666442871094.
Epoch 455: Train Loss = 1.5647938251495361. Val Loss = 1.8985213041305542.
Epoch 456: Train Loss = 1.5617141723632812. Val Loss = 1.898795485496521.
Epoch 457: Train Loss = 1.5

...

Epoch 1540: Train Loss = 1.5073200464248657. Val Loss = 1.8848398923873901.
Epoch 1541: Train Loss = 1.5084645748138428. Val Loss = 1.8848397731781006.
Epoch 1542: Train Loss = 1.5124399662017822. Val Loss = 1.8848398923873901.
Epoch 1543: Train Loss = 1.5042439699172974. Val Loss = 1.8848398923873901.
Epoch 1544: Train Loss = 1.506665825843811. Val Loss = 1.8848398923873901.
Epoch 1545: Train Loss = 1.5096776485443115. Val Loss = 1.8848398923873901.
Epoch 1546: Train Loss = 1.5037912130355835. Val Loss = 1.8848398923873901.
Epoch 1547: Train Loss = 1.5057157278060913. Val Loss = 1.8848400115966797.
Epoch 1548: Train Loss = 1.5062669515609741. Val Loss = 1.8848398923873901.
Epoch 1549: Train Loss = 1.5055958032608032. Val Loss = 1.8848400115966797.
Epoch 1550: Train Loss = 1.5066297054290771. Val Loss = 1.8848398923873901.
Epoch 1551: Train Loss = 1.5130254030227661. Val Loss = 1.8848398923873901.
Epoch 1552: Train Loss = 1.5059081315994263. Val Loss = 1.8848400115966797.
Epoch 1553: T

Epoch 1649: Train Loss = 1.5069221258163452. Val Loss = 1.8848395347595215.
Epoch 1650: Train Loss = 1.5070772171020508. Val Loss = 1.884839415550232.
Epoch 1651: Train Loss = 1.5104591846466064. Val Loss = 1.884839415550232.
Epoch 1652: Train Loss = 1.5084651708602905. Val Loss = 1.884839415550232.
Epoch 1653: Train Loss = 1.509762167930603. Val Loss = 1.884839415550232.
Epoch 1654: Train Loss = 1.5083812475204468. Val Loss = 1.8848392963409424.
Epoch 1655: Train Loss = 1.5067522525787354. Val Loss = 1.884839415550232.
Epoch 1656: Train Loss = 1.5085288286209106. Val Loss = 1.8848392963409424.
Epoch 1657: Train Loss = 1.5056949853897095. Val Loss = 1.8848391771316528.
Epoch 1658: Train Loss = 1.5037424564361572. Val Loss = 1.8848389387130737.
Epoch 1659: Train Loss = 1.5065406560897827. Val Loss = 1.8848391771316528.
Epoch 1660: Train Loss = 1.505700707435608. Val Loss = 1.8848392963409424.
Epoch 1661: Train Loss = 1.50674569606781. Val Loss = 1.8848392963409424.
Epoch 1662: Train Los

Epoch 1758: Train Loss = 1.5065206289291382. Val Loss = 1.8848378658294678.
Epoch 1759: Train Loss = 1.5131405591964722. Val Loss = 1.8848378658294678.
Epoch 1760: Train Loss = 1.5061352252960205. Val Loss = 1.8848375082015991.
Epoch 1761: Train Loss = 1.5092830657958984. Val Loss = 1.8848375082015991.
Epoch 1762: Train Loss = 1.510398030281067. Val Loss = 1.8848376274108887.
Epoch 1763: Train Loss = 1.5101460218429565. Val Loss = 1.8848378658294678.
Epoch 1764: Train Loss = 1.5123652219772339. Val Loss = 1.8848376274108887.
Epoch 1765: Train Loss = 1.5058854818344116. Val Loss = 1.8848373889923096.
Epoch 1766: Train Loss = 1.5062330961227417. Val Loss = 1.88483726978302.
Epoch 1767: Train Loss = 1.5051530599594116. Val Loss = 1.88483726978302.
Epoch 1768: Train Loss = 1.5096378326416016. Val Loss = 1.88483726978302.
Epoch 1769: Train Loss = 1.5073248147964478. Val Loss = 1.8848375082015991.
Epoch 1770: Train Loss = 1.5095977783203125. Val Loss = 1.88483726978302.
Epoch 1771: Train Los

Epoch 1867: Train Loss = 1.5031025409698486. Val Loss = 1.8848360776901245.
Epoch 1868: Train Loss = 1.5103260278701782. Val Loss = 1.8848357200622559.
Epoch 1869: Train Loss = 1.508304476737976. Val Loss = 1.884835958480835.
Epoch 1870: Train Loss = 1.5121583938598633. Val Loss = 1.8848357200622559.
Epoch 1871: Train Loss = 1.5062572956085205. Val Loss = 1.8848357200622559.
Epoch 1872: Train Loss = 1.5122166872024536. Val Loss = 1.8848357200622559.
Epoch 1873: Train Loss = 1.504988670349121. Val Loss = 1.8848357200622559.
Epoch 1874: Train Loss = 1.5080105066299438. Val Loss = 1.884835958480835.
Epoch 1875: Train Loss = 1.5097533464431763. Val Loss = 1.884835958480835.
Epoch 1876: Train Loss = 1.5093395709991455. Val Loss = 1.8848357200622559.
Epoch 1877: Train Loss = 1.5122252702713013. Val Loss = 1.8848356008529663.
Epoch 1878: Train Loss = 1.5071207284927368. Val Loss = 1.8848357200622559.
Epoch 1879: Train Loss = 1.5057709217071533. Val Loss = 1.8848356008529663.
Epoch 1880: Train

Epoch 1976: Train Loss = 1.5065847635269165. Val Loss = 1.8848326206207275.
Epoch 1977: Train Loss = 1.5064427852630615. Val Loss = 1.8848329782485962.
Epoch 1978: Train Loss = 1.5092397928237915. Val Loss = 1.8848329782485962.
Epoch 1979: Train Loss = 1.5102711915969849. Val Loss = 1.8848328590393066.
Epoch 1980: Train Loss = 1.5020198822021484. Val Loss = 1.8848326206207275.
Epoch 1981: Train Loss = 1.5113672018051147. Val Loss = 1.884832501411438.
Epoch 1982: Train Loss = 1.5078270435333252. Val Loss = 1.8848326206207275.
Epoch 1983: Train Loss = 1.5059658288955688. Val Loss = 1.8848326206207275.
Epoch 1984: Train Loss = 1.5042387247085571. Val Loss = 1.884832501411438.
Epoch 1985: Train Loss = 1.5082935094833374. Val Loss = 1.8848326206207275.
Epoch 1986: Train Loss = 1.509076714515686. Val Loss = 1.8848326206207275.
Epoch 1987: Train Loss = 1.5099354982376099. Val Loss = 1.8848323822021484.
Epoch 1988: Train Loss = 1.5109697580337524. Val Loss = 1.8848323822021484.
Epoch 1989: Tra

In [26]:
random_net

40.58379530906677

| Model        | Loss           | Accuracy  |
| ------------- |:-------------:| -----:|
| Random embeddings      | 2.31 | 8.07% |
| Random musiCNN      | 1.88 | 40,58% |
| musiCNN waveform      |    0.67   |   80.50% |
| musiCNN melonMEL | 0.92      |    74.57% |